In [178]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
import operator

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import os 
os.chdir('/Users/deepak/capstone/clutch-baller')
# os.chdir('./clutch-baller')
import practice_code as pc
# os.chdir('..')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
df = pd.read_csv('/Users/deepak/Desktop/NBA_Data/Every_Season_Data/[2005-2006]-combined-stats.csv')
df.head()

,game_id,data_set,date,a1,a2,a3,a4,a5,h1,h2,h3,h4,h5,period,away_score,home_score,remaining_time,elapsed,play_length,play_id,team,event_type,assist,away,home,block,entered,left,num,opponent,outof,player,points,possession,reason,result,steal,type,shot_distance,original_x,original_y,converted_x,converted_y,description
0,"=""0020500001""",2005-2006 Regular Season,2005-11-01,Bobby Simmons,Andrew Bogut,Jamaal Magloire,Michael Redd,T.J. Ford,Kyle Korver,Chris Webber,Steven Hunter,Andre Iguodala,Allen Iverson,1,0,0,00:12:00,00:00:00,00:00:00,0,NaN,start of period,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,start of period,NaN,NaN,NaN,NaN,NaN,NaN
1,"=""0020500001""",2005-2006 Regular Season,2005-11-01,Bobby Simmons,Andrew Bogut,Jamaal Magloire,Michael Redd,T.J. Ford,Kyle Korver,Chris Webber,Steven Hunter,Andre Iguodala,Allen Iverson,1,0,0,00:12:00,00:00:00,00:00:00,1,PHI,jump ball,NaN,Andrew Bogut,Steven Hunter,NaN,NaN,NaN,NaN,NaN,NaN,Steven Hunter,NaN,Allen Iverson,NaN,NaN,NaN,jump ball,NaN,NaN,NaN,NaN,NaN,Jump Ball Hunter vs. Bogut: Tip to Iverson
2,"=""0020500001""",2005-2006 Regular Season,2005-11-01,Bobby Simmons,Andrew Bogut,Jamaal Magloire,Michael Redd,T.J. Ford,Kyle Korver,Chris Webber,Steven Hunter,Andre Iguodala,Allen Iverson,1,0,2,00:11:46,00:00:14,00:00:14,2,PHI,shot,Allen Iverson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chris Webber,2.0,NaN,NaN,made,NaN,Layup,0.0,0.0,0.0,25.0,89.0,Webber Layup (2 PTS) (Iverson 1 AST)
3,"=""0020500001""",2005-2006 Regular Season,2005-11-01,Bobby Simmons,Andrew Bogut,Jamaal Magloire,Michael Redd,T.J. Ford,Kyle Korver,Chris Webber,Steven Hunter,Andre Iguodala,Allen Iverson,1,2,2,00:11:31,00:00:29,00:00:15,3,MIL,shot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Michael Redd,2.0,NaN,NaN,made,NaN,Running Jump Shot,16.0,-143.0,61.0,39.3,11.1,Redd 16' Running Jump Shot (2 PTS)
4,"=""0020500001""",2005-2006 Regular Season,2005-11-01,Bobby Simmons,Andrew Bogut,Jamaal Magloire,Michael Redd,T.J. Ford,Kyle Korver,Chris Webber,Steven Hunter,Andre Iguodala,Allen Iverson,1,2,4,00:11:16,00:00:44,00:00:15,4,PHI,shot,Chris Webber,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Allen Iverson,2.0,NaN,NaN,made,NaN,Jump Shot,16.0,-156.0,-34.0,9.4,92.4,Iverson 16' Jump Shot (2 PTS) (Webber 1 AST)


##Added another column that shows the pts difference between both teams

In [63]:
df = pc.add_pts(df)
df.head()

,game_id,data_set,date,a1,a2,a3,a4,a5,h1,h2,h3,h4,h5,period,away_score,home_score,remaining_time,elapsed,play_length,play_id,team,event_type,assist,away,home,block,entered,left,num,opponent,outof,player,points,possession,reason,result,steal,type,shot_distance,original_x,original_y,converted_x,converted_y,description,pts_difference
0,"=""0020500001""",2005-2006 Regular Season,2005-11-01,Bobby Simmons,Andrew Bogut,Jamaal Magloire,Michael Redd,T.J. Ford,Kyle Korver,Chris Webber,Steven Hunter,Andre Iguodala,Allen Iverson,1,0,0,00:12:00,00:00:00,00:00:00,0,NaN,start of period,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,start of period,NaN,NaN,NaN,NaN,NaN,NaN,0
1,"=""0020500001""",2005-2006 Regular Season,2005-11-01,Bobby Simmons,Andrew Bogut,Jamaal Magloire,Michael Redd,T.J. Ford,Kyle Korver,Chris Webber,Steven Hunter,Andre Iguodala,Allen Iverson,1,0,0,00:12:00,00:00:00,00:00:00,1,PHI,jump ball,NaN,Andrew Bogut,Steven Hunter,NaN,NaN,NaN,NaN,NaN,NaN,Steven Hunter,NaN,Allen Iverson,NaN,NaN,NaN,jump ball,NaN,NaN,NaN,NaN,NaN,Jump Ball Hunter vs. Bogut: Tip to Iverson,0
2,"=""0020500001""",2005-2006 Regular Season,2005-11-01,Bobby Simmons,Andrew Bogut,Jamaal Magloire,Michael Redd,T.J. Ford,Kyle Korver,Chris Webber,Steven Hunter,Andre Iguodala,Allen Iverson,1,0,2,00:11:46,00:00:14,00:00:14,2,PHI,shot,Allen Iverson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chris Webber,2.0,NaN,NaN,made,NaN,Layup,0.0,0.0,0.0,25.0,89.0,Webber Layup (2 PTS) (Iverson 1 AST),-2
3,"=""0020500001""",2005-2006 Regular Season,2005-11-01,Bobby Simmons,Andrew Bogut,Jamaal Magloire,Michael Redd,T.J. Ford,Kyle Korver,Chris Webber,Steven Hunter,Andre Iguodala,Allen Iverson,1,2,2,00:11:31,00:00:29,00:00:15,3,MIL,shot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Michael Redd,2.0,NaN,NaN,made,NaN,Running Jump Shot,16.0,-143.0,61.0,39.3,11.1,Redd 16' Running Jump Shot (2 PTS),0
4,"=""0020500001""",2005-2006 Regular Season,2005-11-01,Bobby Simmons,Andrew Bogut,Jamaal Magloire,Michael Redd,T.J. Ford,Kyle Korver,Chris Webber,Steven Hunter,Andre Iguodala,Allen Iverson,1,2,4,00:11:16,00:00:44,00:00:15,4,PHI,shot,Chris Webber,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Allen Iverson,2.0,NaN,NaN,made,NaN,Jump Shot,16.0,-156.0,-34.0,9.4,92.4,Iverson 16' Jump Shot (2 PTS) (Webber 1 AST),-2


In [64]:
col_to_remove = ['a1', 'a2', 'a3','a4','a5','h1','h2','h3','h4','h5', 'play_length', 'entered', 'left', 'possession', 'shot_distance', 'original_x', 'original_y', 'converted_x', 'converted_y', 'num', 'away', 'home', 'outof', 'opponent', 'reason', 'elapsed', 'play_id']

##Remove unnecessary columns

In [65]:
df = pc.remove_col(df, col_to_remove)
df.head()

,game_id,data_set,date,period,away_score,home_score,remaining_time,team,event_type,assist,block,player,points,result,steal,type,description,pts_difference
0,"=""0020500001""",2005-2006 Regular Season,2005-11-01,1,0,0,00:12:00,NaN,start of period,NaN,NaN,NaN,NaN,NaN,NaN,start of period,NaN,0
1,"=""0020500001""",2005-2006 Regular Season,2005-11-01,1,0,0,00:12:00,PHI,jump ball,NaN,NaN,Steven Hunter,NaN,NaN,NaN,jump ball,Jump Ball Hunter vs. Bogut: Tip to Iverson,0
2,"=""0020500001""",2005-2006 Regular Season,2005-11-01,1,0,2,00:11:46,PHI,shot,Allen Iverson,NaN,Chris Webber,2.0,made,NaN,Layup,Webber Layup (2 PTS) (Iverson 1 AST),-2
3,"=""0020500001""",2005-2006 Regular Season,2005-11-01,1,2,2,00:11:31,MIL,shot,NaN,NaN,Michael Redd,2.0,made,NaN,Running Jump Shot,Redd 16' Running Jump Shot (2 PTS),0
4,"=""0020500001""",2005-2006 Regular Season,2005-11-01,1,2,4,00:11:16,PHI,shot,Chris Webber,NaN,Allen Iverson,2.0,made,NaN,Jump Shot,Iverson 16' Jump Shot (2 PTS) (Webber 1 AST),-2


##df showing all the games within 6 points when two min left in 4th

In [73]:
clutch_time = pc.clutch_moment(df)
clutch_time.head()

,game_id,data_set,date,period,away_score,home_score,remaining_time,team,event_type,assist,block,player,points,result,steal,type,description,pts_difference
443,"=""0020500001""",2005-2006 Regular Season,2005-11-01,4,96,102,00:01:07,MIL,free throw,NaN,NaN,Michael Redd,1.0,made,NaN,Free Throw 1 of 2,Redd Free Throw 1 of 2 (20 PTS),-6
444,"=""0020500001""",2005-2006 Regular Season,2005-11-01,4,96,102,00:01:07,PHI,sub,NaN,NaN,Andre Iguodala,NaN,NaN,NaN,sub,SUB: Korver FOR Iguodala,-6
445,"=""0020500001""",2005-2006 Regular Season,2005-11-01,4,97,102,00:01:07,MIL,free throw,NaN,NaN,Michael Redd,1.0,made,NaN,Free Throw 2 of 2,Redd Free Throw 2 of 2 (21 PTS),-5
446,"=""0020500001""",2005-2006 Regular Season,2005-11-01,4,97,102,00:00:47,PHI,miss,NaN,NaN,Allen Iverson,0.0,missed,NaN,Jump Shot,MISS Iverson 18' Jump Shot,-5
447,"=""0020500001""",2005-2006 Regular Season,2005-11-01,4,97,102,00:00:45,MIL,rebound,NaN,NaN,T.J. Ford,NaN,NaN,NaN,rebound defensive,Ford REBOUND (Off:1 Def:6),-5


##Remove Rows that are not needed (such as fouls, subs, timeouts etc)

In [74]:
rows = ['sub', 'timeout', 'uknown', 'unknown', 'Ejection', 'violat', 'traveling', 'team reb', 'foul',
        'turnover', 'of period', 'lost ball', 'goaltend', 'bad pass', 'illegal', 'jump ball', 'o']

In [75]:
clutch_time = pc.remove_junk_rows(clutch_time, 'type', rows)
clutch_time.head()

,game_id,data_set,date,period,away_score,home_score,remaining_time,team,event_type,assist,block,player,points,result,steal,type,description,pts_difference
443,"=""0020500001""",2005-2006 Regular Season,2005-11-01,4,96,102,00:01:07,MIL,free throw,NaN,NaN,Michael Redd,1.0,made,NaN,Free Throw 1 of 2,Redd Free Throw 1 of 2 (20 PTS),-6
445,"=""0020500001""",2005-2006 Regular Season,2005-11-01,4,97,102,00:01:07,MIL,free throw,NaN,NaN,Michael Redd,1.0,made,NaN,Free Throw 2 of 2,Redd Free Throw 2 of 2 (21 PTS),-5
446,"=""0020500001""",2005-2006 Regular Season,2005-11-01,4,97,102,00:00:47,PHI,miss,NaN,NaN,Allen Iverson,0.0,missed,NaN,Jump Shot,MISS Iverson 18' Jump Shot,-5
447,"=""0020500001""",2005-2006 Regular Season,2005-11-01,4,97,102,00:00:45,MIL,rebound,NaN,NaN,T.J. Ford,NaN,NaN,NaN,rebound defensive,Ford REBOUND (Off:1 Def:6),-5
448,"=""0020500001""",2005-2006 Regular Season,2005-11-01,4,99,102,00:00:41,MIL,shot,NaN,NaN,T.J. Ford,2.0,made,NaN,Driving Layup,Ford Driving Layup (14 PTS),-3


##Remove game that didnt have clutch moment but got close for a sec

In [87]:
clutch_time = pc.remove_not_imp_games(clutch_time, 'game_id')
clutch_time['game_id'].value_counts().count()

533

##Separate playoff games from regular season games

In [580]:
regular_season, playoffs =  pc.regular_or_playoffs(clutch_time, 'data_set')

In [581]:
regular_season.head()

,game_id,data_set,date,period,away_score,home_score,remaining_time,team,event_type,assist,block,player,points,result,steal,type,description,pts_difference
443,"=""0020500001""",2005-2006 Regular Season,2005-11-01,4,96,102,00:01:07,MIL,free throw,NaN,NaN,Michael Redd,1.0,made,NaN,Free Throw 1 of 2,Redd Free Throw 1 of 2 (20 PTS),-6
445,"=""0020500001""",2005-2006 Regular Season,2005-11-01,4,97,102,00:01:07,MIL,free throw,NaN,NaN,Michael Redd,1.0,made,NaN,Free Throw 2 of 2,Redd Free Throw 2 of 2 (21 PTS),-5
446,"=""0020500001""",2005-2006 Regular Season,2005-11-01,4,97,102,00:00:47,PHI,miss,NaN,NaN,Allen Iverson,0.0,missed,NaN,Jump Shot,MISS Iverson 18' Jump Shot,-5
447,"=""0020500001""",2005-2006 Regular Season,2005-11-01,4,97,102,00:00:45,MIL,rebound,NaN,NaN,T.J. Ford,NaN,NaN,NaN,rebound defensive,Ford REBOUND (Off:1 Def:6),-5
448,"=""0020500001""",2005-2006 Regular Season,2005-11-01,4,99,102,00:00:41,MIL,shot,NaN,NaN,T.J. Ford,2.0,made,NaN,Driving Layup,Ford Driving Layup (14 PTS),-3


In [90]:
playoffs.head()

,game_id,data_set,date,period,away_score,home_score,remaining_time,team,event_type,assist,block,player,points,result,steal,type,description,pts_difference
551712,"=""0040500111""",2006 Playoff,2006-04-22,4,104,106,00:01:49,MIA,free throw,NaN,NaN,Dwyane Wade,1.0,made,NaN,Free Throw 1 of 2,Wade Free Throw 1 of 2 (27 PTS),-2
551714,"=""0040500111""",2006 Playoff,2006-04-22,4,104,107,00:01:49,MIA,free throw,NaN,NaN,Dwyane Wade,1.0,made,NaN,Free Throw 2 of 2,Wade Free Throw 2 of 2 (28 PTS),-3
551716,"=""0040500111""",2006 Playoff,2006-04-22,4,104,107,00:01:21,MIA,miss,NaN,NaN,Antoine Walker,0.0,missed,NaN,Jump Shot,MISS Walker 8' Jump Shot,-3
551717,"=""0040500111""",2006 Playoff,2006-04-22,4,104,107,00:01:19,CHI,rebound,NaN,NaN,Andres Nocioni,NaN,NaN,NaN,rebound defensive,Nocioni REBOUND (Off:2 Def:13),-3
551718,"=""0040500111""",2006 Playoff,2006-04-22,4,104,107,00:01:00,CHI,miss,NaN,NaN,Kirk Hinrich,0.0,missed,NaN,Turnaround Jump Shot,MISS Hinrich 17' Turnaround Jump Shot,-3


##Shows the shooting stats for a player, righ now showing for last 2 min but can change df to get full season or playoffs!

In [91]:
stats = pc.get_percent(regular_season, 'player', 'event_type', 'Steve Nash')
stats

('Shooting Percentage:0.377', 'Made: 20', 'Attempt: 53')

##Free throw numbers

In [92]:
free_throw = pc.free_throw_percent(regular_season, 'player', 'event_type', 'result', 'Steve Nash')
free_throw

('Free Throw:0.882', 'Made: 30', 'Attempt: 34')

##Compare shooting stats between 2 players

In [93]:
one_on_one = pc.compare_stats(regular_season, 'player', 'event_type', 'Kobe Bryant', "Shaquille O'Neal")
one_on_one

('Player A:Kobe Bryant',
 'Shooting Percentage:0.372',
 'Made: 32',
 'Attempt: 86',
 "Player B:Shaquille O'Neal",
 'Shooting Percentage:0.615',
 'Made: 8',
 'Attempt: 13')

##Compares Shooting and Free Throws

In [94]:
full_comparison = pc.compare_players(regular_season, 'player', 'event_type', 'result','Kobe Bryant', 'Steve Nash')
full_comparison

(('Player A:Kobe Bryant',
  'Shooting Percentage:0.372',
  'Made: 32',
  'Attempt: 86',
  'Free Throw:0.871',
  'Made: 54',
  'Attempt: 62'),
 ('Player B:Steve Nash',
  'Shooting Percentage:0.377',
  'Made: 20',
  'Attempt: 53',
  'Free Throw:0.882',
  'Made: 30',
  'Attempt: 34'))

In [531]:
#All the rows with offensive rebounds for all the players
rebound_type = regular_season[regular_season['type']=='rebound offensive']


In [532]:
dic= {}
for value in rebound_type['player'].values:
    if value not in dic.keys():
        dic[value]=1
    else:
        dic[value]+=1

In [533]:
#counts how many rebounds each player has in total
rebound_per_player = pd.DataFrame.from_dict(dic,orient = 'index')
rebound_per_player = rebound_per_player.reset_index()
rebound_per_player = rebound_per_player.rename(columns={'index':'player', 0:'total_rebound'})


#take the number of games played from regular season data so we can get the actual average
r= (regular_season.groupby(['player', 'game_id']).count()).reset_index()
r = (r.groupby('player').count()).reset_index()
r = r[['player','game_id']]

j = r.merge(rebound_per_player, on = 'player')

#create a new column with off_rebund per game
j['rebound_per_clutch_time']=j.apply(lambda row: row.total_rebound / row.game_id, axis =1)

#sorted by amount of games and total rebounds
j = j.sort_values(by = ['total_rebound','game_id','rebound_per_clutch_time'], ascending=False)


In [534]:
j.head()

,player,game_id,total_rebound,rebound_per_clutch_time
75,Dwight Howard,26,14,0.538462
43,Chris Bosh,25,14,0.560000
42,Charlie Villanueva,26,13,0.500000
244,Vince Carter,23,13,0.565217
233,Tim Duncan,22,13,0.590909


#assists

In [351]:
#is_assist['assist'].replace('', np.nan, inplace=True)

In [590]:
#DF with all the plays that had a assist
is_assist = regular_season[regular_season['assist']!='NaN']
is_assist.dropna(subset=['assist'], inplace=True)

In [404]:
#all the assist are grouoed by player names and shows how many assists
#they had in each game (only show games where they had a assist)
z = (is_assist.groupby(['assist', 'game_id']).count()).reset_index()

#takes all the players from z and add all all the games they apperaed in which they had a assist
y = (z.groupby('assist').count()).reset_index()
y = y[['assist', 'game_id']]

In [406]:
assist_dic= {}
for value in is_assist['assist'].values:
    if value not in assist_dic.keys():
        assist_dic[value]=1
    else:
        assist_dic[value]+=1

In [546]:
#counts how many rebounds each player has in total
assist_per_player = pd.DataFrame.from_dict(assist_dic,orient = 'index')
assist_per_player = assist_per_player.reset_index()
assist_per_player = assist_per_player.rename(columns={'index':'assist', 0:'total_assist'})
assist_per_player = assist_per_player.rename(columns={'assist':'player'})


r= (regular_season.groupby(['player', 'game_id']).count()).reset_index()
r = (r.groupby('player').count()).reset_index()
r = r[['player','game_id']]

l = r.merge(assist_per_player, on = 'player')



#create a new column with assists per game

l['assist_per_clutch_time']=l.apply(lambda row: row.total_assist / row.game_id, axis =1)

#sorted by amount of games and assists

l = l.sort_values(by = ['total_assist', 'game_id'], ascending=False)

##Blocks

In [561]:
blocks = regular_season[regular_season['block']!='NaN']

In [563]:
blk_dic= {}
for value in blocks['block'].values:
    if value not in blk_dic.keys():
        blk_dic[value]=1
    else:
        blk_dic[value]+=1

In [585]:
blocks_1 = pd.DataFrame.from_dict(blk_dic,orient = 'index')
blocks_1 = blocks_1.reset_index()
blocks_1 = blocks_1.rename(columns={'index':'block', 0:'total_blocks'})
blocks_1 = blocks_1.rename(columns={'block':'player'})


r= (regular_season.groupby(['player', 'game_id']).count()).reset_index()
r = (r.groupby('player').count()).reset_index()
r = r[['player','game_id']]

l = r.merge(blocks_1, on = 'player')



#create a new column with assists per game

l['block_per_clutch_time']=l.apply(lambda row: row.total_blocks / row.game_id, axis =1)

#sorted by amount of games and assists

l = l.sort_values(by = ['total_blocks', 'game_id'], ascending=False)

In [586]:
l.head()

,player,game_id,total_blocks,block_per_clutch_time
94,Marcus Camby,16,14,0.875000
8,Andrei Kirilenko,19,13,0.684211
76,Josh Smith,30,11,0.366667
45,Dirk Nowitzki,31,8,0.258065
142,Tyson Chandler,16,8,0.500000
